In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('mbti_tfidf.csv', encoding = 'utf-8')

In [3]:
df_x_train = df.drop('label_type', axis = 1)
df_y_train = df['label_type']

In [4]:
# 讀取 linkin 貼文 tf-idf 資料集以及職業的資料集

df_linkin_tfidf = pd.read_csv('LinkedIn_tfidf_new.csv', encoding = 'utf-8')

In [5]:
df_linkin_tfidf.head()

,name,headline,career_keywords,0,0.1,00am,00pm,1,5,7880,...,????????.52,????????.53,??????????.48,????????.54,??????????.49,??????????.50,????????.55,??????????.51,????????????.19,????????.56
0,Katie Martell,"""To pay attention, this is our endless and pro...","B2B, Marketing, Business, Social justice",0.010267,0.006207,0.0,0.027836,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Gary Frisch,30-Year Public Relations Pro and Skilled Writer,"Public Relations, Writer, Journalist, Healthcare",0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,String Nguyen,4x LinkedIn Top Voice ???? Follow for The Trus...,"Brand Marketing, Brand Ambassador",0.001021,0.007922,0.0,0.000000,0.0,0.00434,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Michelle de Haaff,Advisor to Healthcare Tech cos.\nAssistant Dir...,"Healthcare Advisor , Assistant Director, Biode...",0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Justin Bariso,Author at EQ Applied | Management Thinker an...,"Author, Management, Emotions Study",0.004744,0.006692,0.0,0.000000,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# df_linkin_tfidf 欄位處理 (字的處理)

# training data 的字
terms = [c for c in df_x_train.columns]

# df_linkin_tfidf 的字
linkin_terms = [c for c in df_linkin_tfidf.columns]


# new df_linkin_tfidf
new_df_linkin_tfidf = pd.DataFrame(columns = terms)
for term in terms:
    if term in linkin_terms:
        new_df_linkin_tfidf[term] = df_linkin_tfidf[term].values
    else:
        new_df_linkin_tfidf[term] = [0 for i in range(len(df_linkin_tfidf))]

In [7]:
new_df_linkin_tfidf.head()

,00,000,00100000,01,10,100,1000,101,10char,10th,...,zelda,zen,zero,zodiac,zombie,zombies,zone,zoned,zoning,zoo
0,0,0,0,0,0.022637,0.019780,0.003920,0.004426,0,0.004666,...,0,0.000000,0.000000,0,0,0,0.002726,0,0,0.0
1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,...,0,0.000000,0.000000,0,0,0,0.000000,0,0,0.0
2,0,0,0,0,0.011977,0.004820,0.010916,0.013864,0,0.000000,...,0,0.004621,0.006642,0,0,0,0.005693,0,0,0.0
3,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,...,0,0.000000,0.000000,0,0,0,0.000000,0,0,0.0
4,0,0,0,0,0.007530,0.000914,0.000000,0.000000,0,0.000000,...,0,0.000000,0.001259,0,0,0,0.000000,0,0,0.0


In [9]:
# 使用 SVM (linear)、不做調參與特徵選擇

from sklearn.svm import SVC

SVM_model = SVC(kernel='linear')
SVM_model.fit(df_x_train, df_y_train)

SVC(kernel='linear')

In [68]:
from xgboost import XGBClassifier

labels = [l for l in set(df_y_train)]
indexes = {}
i = 0
for label in labels:
    indexes[label] = i
    i += 1

y_labels = [indexes[label] for label in df_y_train.values]

In [69]:
xgb_model = XGBClassifier()
xgb_model.fit(df_x_train, y_labels)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [70]:
predicted_results = xgb_model.predict(new_df_linkin_tfidf)

In [71]:
mbti_predicted_results = [labels[i] for i in predicted_results]

mbti_predicted_results

['INFJ',
 'INFP',
 'INFJ',
 'INFP',
 'INFJ',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFJ',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFJ',
 'INFJ',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INTJ',
 'INFP',
 'INFJ',
 'INFJ',
 'INFJ',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFJ',
 'INFP',
 'INFP',
 'INFJ',
 'INFP',
 'INFP',
 'INFJ',
 'INFP',
 'INFJ',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INFJ',
 'INFP',
 'INFP',
 'INFP',
 'INFP',
 'INTP',
 'INFP',
 'INFP',
 'INTP',
 'INFJ',
 'INFP',
 'INFP',
 'INFJ',
 'INFJ',
 'INFJ',
 'INFP',
 'INFP',
 'INFP']

In [72]:
# 讀取 df_linkin_tfidf 中各個重要人物的職業

def seperate(text):
    careers = text.split(', ')
#     return [career.lower() for career in careers]
    results = []
    for career in careers:
        temp = career.split(' ')
        for a in temp:
            if len(a) > 0:
                results.append(a.lower())
    return results 
    
df_figure_careers = df_linkin_tfidf[['name', 'career_keywords']].copy()
df_figure_careers['career_keywords'] = df_figure_careers['career_keywords'].apply(seperate)

In [73]:
df_figure_careers['MBTI type'] = mbti_predicted_results

df_figure_careers

,name,career_keywords,MBTI type
0,Katie Martell,"[b2b, marketing, business, social, justice]",INFJ
1,Gary Frisch,"[public, relations, writer, journalist, health...",INFP
2,String Nguyen,"[brand, marketing, brand, ambassador]",INFJ
3,Michelle de Haaff,"[healthcare, advisor, assistant, director, bio...",INFP
4,Justin Bariso,"[author, management, emotions, study]",INFJ
...,...,...,...
64,Tai T.,"[social, impact, leader]",INFJ
65,Amy Blaschka,"[writer, ghostwriter, communications, leadership]",INFJ
66,Daniel Cohen-I'm Flyering,"[talent, brand, hr, recruiter]",INFP
67,Nir Eyal,"[author, public, speaker, consultant]",INFP


In [74]:
df_figure_careers.to_csv('predict_career_outcome.csv',encoding='utf-8')

In [75]:
# 讀取各型人格適合的職業
df_mbti_career = pd.read_csv('MBTI_career.csv', encoding = 'utf-8')
df_mbti_career_seperated = df_mbti_career.copy()
df_mbti_career_seperated['Career'] = df_mbti_career_seperated['Career'].apply(seperate)
df_mbti_career_seperated

,MBTI,Career
0,ENFJ,"[counselor, sales, manager, hr, art, director,..."
1,ENFP,"[reporter, editor, musician, product, manager,..."
2,ENTJ,"[business, administrator, public, relations, s..."
3,ENTP,"[attorney, copywriter, financial, planner, psy..."
4,ESFJ,"[office, manager, technical, support, speciali..."
5,ESFP,"[event, planner, professional, entertainer, sa..."
6,ESTJ,"[judge, coach, financial, officer, hotel, mana..."
7,ESTP,"[firefighter, paramedic, creative, director, p..."
8,INFJ,"[consulting, counselor, coun, writer, scientis..."
9,INFP,"[copywriter, hr, manager, therapist, mental, h..."


In [76]:
# 比對每位重要人員所預測的 MBTI 的職業

num_of_people = len(df_linkin_tfidf)
count = 0
predicted_right_figures = []

for name in df_figure_careers['name'].values:
    figure_careers = df_figure_careers[df_figure_careers['name'] == name]['career_keywords'].values[0]
    mbti = df_figure_careers[df_figure_careers['name'] == name]['MBTI type'].values[0]
    mbti_careers = df_mbti_career_seperated[df_mbti_career_seperated['MBTI'] == mbti].Career.values[0]
    
    for career in figure_careers:
        if career in mbti_careers:
            count += 1
            predicted_right_figures.append(name)
            

In [77]:
set(predicted_right_figures)

{'Amy Blaschka',
 'Barry W. Enderwick',
 'Ben Casselman',
 'Bernard Marr',
 "Daniel Cohen-I'm Flyering",
 'Danielle Newnham',
 'Gary Frisch',
 'Gillian Zoe Segal',
 'James Calder',
 'Jonathan Wolfer',
 'Julie Kliger',
 'Karen Gross',
 'Katie Martell',
 'Lynne Everatt',
 'Marianne Griebler',
 'Michelle Chaffee',
 'Neil Hughes',
 'Nicholas Wyman',
 'Nir Eyal',
 'Quentin Michael Allums',
 'Shama Hyder',
 'Simon Sinek',
 'Stephanie C. O.',
 'Steve Blakeman',
 'Tai T.',
 'Tamal Bandyopadhyay',
 'Tom Foremski',
 'Tom Goodwin',
 'Vikram Mansharamani'}

In [78]:
len(set(predicted_right_figures))

29

In [79]:
len(set(predicted_right_figures)) / num_of_people

0.42028985507246375

# 輸入介面

In [45]:
import tkinter as tk

In [62]:
def callbackW():
    print('yes')

def show():
    request = en.get()
    zE.set(request)
    win.mainloop()

win = tk.Tk()

win.geometry("900x600+300+100")
win.title('MBTI Predictor')

zE = tk.StringVar()
label = tk.Label(win,text = "Enter your post here").pack()
en = tk.StringVar()
enter = tk.Entry(win,text=en).pack()
btn = tk.Button(win,text="Enter",padx =2, pady= 2,command =show).pack()
zE.trace("w", callbackW)

global cv
cv = tk.Canvas()
cv.pack()

win.mainloop()